# `System.Text.Json` and Newtonsoft’s Json.NET

## accessing arrays

Here is a comparison of how we access a JSON array with `JObject` (from Newtonsoft), `JsonDocument` and `JsonNode`.

## notebok setup

In [1]:
#r "nuget: Newtonsoft.Json"

Installed Packages Newtonsoft.Json, 13.0.3

In [2]:
var json = """
{
    "mySet": [
        "one",
        "two",
        "three"
    ]
}
""";

In [3]:
using System.Text.Json;
using System.Text.Json.Nodes;

using Newtonsoft.Json;
using Newtonsoft.Json.Linq;

## parsing the JSON

In [4]:
JObject newtonJObject = JObject.Parse(json);

JsonDocument jsonDocument = JsonDocument.Parse(json);
JsonNode jsonObject = JsonNode.Parse(json);

### What is the difference between `JsonNode` and `JsonDocument`?

Use `JsonDocument` for high-performance, read-only operations. `JsonNode` is meant for read-write operations. `JsonNode` is almost like the Newtonsoft `JToken`.

## accessing the array

Newtonsoft uses an optimistic indexer design:

In [5]:
JToken jToken = newtonJObject["mySet"];

jToken.Type

Array

Microsoft’s `JsonElement` has an optimistic `GetProperty` method:

In [6]:
jsonDocument.RootElement.GetProperty("mySet").ValueKind

Array

But it also has a pessimistic (cautious/consumer-friendly) `TryGetProperty` method:

In [7]:
jsonDocument.RootElement.TryGetProperty("mySet", out JsonElement myArrayElement)

True

After logic-gating the array access, we can now be responsibly optimistic:

In [8]:
myArrayElement.ValueKind

Array

The `JsonNode` approach also has a cautious method, `GetValueKind()`, that can be used when needed: 

In [9]:
JsonNode myArrayNode = jsonObject.AsObject()["mySet"];

myArrayNode.GetValueKind()

Array

## modifying an array

In [10]:
string myJson = """
{
    "kind": "my#things",
    "items": [
        {
            "kind": "my#thing",
            "id": "1"
        },
        {
            "kind": "my#thing",
            "id": "2"
        },
        {
            "kind": "my#thing",
            "id": "3"
        }
    ]
}
""";

string myOtherJson = """
{
    "kind": "my#other-things",
    "items": [
        {
            "kind": "my#other-thing",
            "id": "1",
            "otherProperty": true
        },
        {
            "kind": "my#other-thing",
            "id": "2",
            "otherProperty": false
        },
        {
            "kind": "my#other-thing",
            "id": "3",
            "otherProperty": true
        }
    ]
}
""";

In [15]:
#!csharp
#nullable enable

JsonObject myJsonObject = JsonNode.Parse(myJson).AsObject();
JsonObject myOtherJsonObject = JsonNode.Parse(myOtherJson).AsObject();

JsonArray editingArray = myJsonObject["items"].AsArray();

foreach(JsonObject jO in editingArray.OfType<JsonObject>())
{
    JsonObject jO_other = myOtherJsonObject["items"]
        .AsArray()
        .First(o => o["id"].GetValue<string>() == jO["id"].GetValue<string>())
        .AsObject();

     foreach (KeyValuePair<string, JsonNode?> pair in jO_other)
     {
        jO[pair.Key] = pair.Value.DeepClone();
     }
}

myJsonObject.ToJsonString(new JsonSerializerOptions { WriteIndented = true })

{
  "kind": "my#things",
  "items": [
    {
      "kind": "my#other-thing",
      "id": "1",
      "otherProperty": true
    },
    {
      "kind": "my#other-thing",
      "id": "2",
      "otherProperty": false
    },
    {
      "kind": "my#other-thing",
      "id": "3",
      "otherProperty": true
    }
  ]
}

[Bryan Wilhite is on LinkedIn](https://www.linkedin.com/in/wilhite)🇺🇸💼
